In [278]:
%pylab inline
from IPython.display import Audio
import librosa
import scipy as sp
from numba import jit
from scipy.spatial import distance

figsize(20,6)
prefix="baseline"
def filepre(nm):
    return "tmp/"+prefix+"_"+nm
from IPython.display import clear_output

Populating the interactive namespace from numpy and matplotlib


/home/chrisk/env3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [279]:
import tensorflow as tf
tf.enable_eager_execution()

In [280]:
from tensorflow.python.client import device_lib
tfdevice='/device:GPU:0'
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10772298712735263252, name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1256786126154002957
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16874224058943071659
 physical_device_desc: "device: XLA_CPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10740894925
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 10338563631680231163
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10913290650
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }

In [281]:
sr = 22050
def nrmse(output,target):
    assert(output.shape[0] == target.shape[0])
    combinedVar = 0.5 * (np.var(target, ddof=1) + np.var(output, ddof=1))
    errorSignal = output - target
    return np.sqrt(np.mean(errorSignal ** 2) / combinedVar)

def generateInternalWeights(nInternalUnits, connectivity):
    success = False
    internalWeights = 0
    while success == False:
        try:
            internalWeights = np.random.randn(nInternalUnits,nInternalUnits) * (np.random.random((nInternalUnits,nInternalUnits)) < connectivity)
            specRad = max(abs(np.linalg.eig(internalWeights)[0]))
            if (specRad > 0):
                internalWeights = internalWeights / specRad
                success = True
        except e:
            print(e)
    return internalWeights

pLoop = lambda n,p: p[n%p.size]

In [282]:
from datetime import datetime
import os
resultsFolderName = "csynthResults/results" + str(datetime.now()) + "/"
os.mkdir(resultsFolderName)


In [283]:
relevant_path = "audios/ixi"
ixiFiles = [fn for fn in os.listdir(relevant_path)
              if fn.endswith('wav')]
# ixiFiles = ['909a_22k.wav']
print(ixiFiles)


['zhish.wav', 'kernelmix.wav', 'rocks.wav', 'boomwag.wav', 'snork.wav', 'insec3.wav', 'MacrosemiaTonk.wav', 'pattern10.wav', 'MeimunaNau.wav', 'backswing-old1.wav', 'audoubelclick2.wav', 'laboa.wav', 'swipe-old1.wav', 'Macrosemia.wav', 'revbell.wav', 'laekur.wav', 'birta.wav', 'clicks1.wav', 'clicko-old1.wav', 'bellx.wav', 'camina2.wav', 'dentist-old1.wav', 'heart.wav', 'electro.wav', 'bello.wav', 'camina1.wav', 'noinoi.wav', 'click.wav', 'InsectInfestationZG.wav', 'harshi-old1.wav', 'bellrip3.wav', 'iron.wav', 'patterndrone.wav', 'ausiclick.wav', 'knock.wav', 'rotatingIron.wav', 'firespark.wav', 'insec2.wav', 'camina4.wav', 'kicic.wav', 'phoo-old1.wav', 'bellrip2.wav', 'dalispark.wav', 'triplet.wav', 'glitch-old1.wav', 'ice.wav', 'Cryptotympana.wav', 'vindvabd.wav', 'pattern11.wav', 'camina3.wav', 'paper.wav', 'InsectFly.wav', 'skake2Ed.wav', 'kicicboom.wav', 'bellrip.wav', 'holeMONO.wav', 'spark.wav', 'elstatic-old1.wav', 'jump.wav', 'drr.wav', 'boom2.wav', 'flash-old1.wav', 'pattern

In [284]:
def ixistr(id):
    return str(id) + "_" + ixiFiles[id] + "_"

In [285]:
def compareMFCCs(seq1, seq2):
    fftSize=2048
    hop=64    
    melspec = librosa.feature.melspectrogram(y=seq1, sr=sr, n_fft=fftSize,hop_length=hop)
    mfccs = librosa.feature.mfcc(S=melspec,n_mfcc=20)[1:,:]
    melspec2 = librosa.feature.melspectrogram(y=seq2, sr=sr, n_fft=fftSize,hop_length=hop)
    mfccs2 = librosa.feature.mfcc(S=melspec2,n_mfcc=20)[1:,:]    
    return nrmse(mfccs.flatten(), mfccs2.flatten())

In [319]:
def computeConceptor(p, net, i_pattern, alpha):
    print('Computing conceptor, alpha: ', alpha)
#     Cs = np.zeros((4, 1), dtype=np.object)
    R = net['patternRs'][0,i_pattern]
    [U,s,V] = svd(R)
#     s = svd(R, compute_uv=False)
    S = tf.diag(s)
#     Snew = (S.dot(linalg.inv(S + pow(alpha, -2) * np.eye(p['N']))))

#     C =  U.dot(Snew).dot(U.T);
    sinv = tf.matrix_inverse(tf.add(S, tf.multiply(double(pow(alpha, -2)), tf.eye(p['N'], dtype=float64))))
    Snew = tf.matmul(S,sinv)
#     Snew = tf.matmul(Snew, tf.eye(p['N'], dtype=float64))
#     Snew = Snew.numpy()
#     Snew = (S * linalg.inv(S + pow(alpha, -2) * np.eye(p['N'])))
    tfU = tf.constant(U)
    C = tf.matmul(tfU,Snew)
    C = tf.matmul(C,tfU, adjoint_b=True)
    return C


def testConceptor(p, C, net, recallTestLength, tfW, tfWbias):
    with tf.device(tfdevice):    
        trials = 1
        attens = np.zeros(trials)
        LR = array(p['LR'])
        LROneMinus = array(1.0 - p['LR'])
        tfLR = tf.constant(LR)
        tfLROneMinus = tf.constant(LROneMinus)
#         tfWbias = tf.constant(net['Wbias'])
#         tfW = tf.constant(net['W'])
#         tfC = tf.constant(C)
        for i_trial in range(trials):
            x_CTestPL = np.zeros((p['N'], recallTestLength))
            z_CTestPL = np.zeros((p['N'], recallTestLength))
#             tfx_CTestPL = tf.TensorArray(tfW.dtype,p['N'])
#             tfz_CTestPL = tf.TensorArray(tfW.dtype,p['N'])
            x = tf.constant(0.5 * np.random.randn(p['N'],1))
            for n in range(recallTestLength + p['washoutLength']):
                xOld = tf.constant(x)
                Wtarget = tf.matmul(tfW, x)
                leakTerm = tf.multiply(LROneMinus,xOld)
                newX =tf.tanh(tf.add(Wtarget, tfWbias))
                newXLeaked = tf.multiply(LR,newX)
                z = tf.add(leakTerm,newXLeaked)
                x = tf.matmul(C,z)
                if (n > p['washoutLength']):
#                     tfx_CTestPL.write(n-p['washoutLength'], tf.transpose(x))
#                     tfz_CTestPL.write(n-p['washoutLength'], tf.transpose(z))
                    x_CTestPL[:,n-p['washoutLength']] = tf.transpose(x).numpy()
                    z_CTestPL[:,n-p['washoutLength']] = tf.transpose(z).numpy()
#             x_CTestPL = tfx_CTestPL.gather(tf.range(0,recallTestLength,1))
#             z_CTestPL = tfz_CTestPL.gather(tf.range(0,recallTestLength,1))
            attenuation = np.mean(pow(np.linalg.norm(z_CTestPL[:,:] - x_CTestPL[:,:], axis=1),2)) / np.mean(pow(np.linalg.norm(z_CTestPL[:,:], axis=1),2))
            attens[i_trial] = attenuation

    return np.mean(attens)


In [288]:
def makeLoadedNetwork_v2(p):
    Netconnectivity = 1
    if p['N'] > 20:
        Netconnectivity = 10.0/p['N'];
    WstarRaw = generateInternalWeights(p['N'], Netconnectivity)
    WinRaw = 2 * (np.random.rand(p['N'], 1) - 0.5)
    WbiasRaw = 2 * (np.random.rand(p['N'], 1) - 0.5)

    #Scale raw weights     
    Wstar = p['NetSR'] * WstarRaw;
    Win = p['NetinpScaling'] * WinRaw;
    Wbias = p['BiasScaling'] * WbiasRaw;  
    I = np.eye(p['N'])
    x = np.zeros((p['N'],1))
    
    allTrainxArgs = np.zeros((p['N'] + 1, 0));
    allTrainOldxArgs = np.zeros((p['N'], 0));
    allTrainWtargets = np.zeros((p['N'], 0));
    allTrainOuts = np.zeros((1, 0));
    patternRs =  np.zeros((1, p['patts'].shape[0]), dtype=np.object)
    print('Loading patterns: ', end='')
    LR = array(p['LR'])
    LROneMinus = array(1.0 - p['LR'])
    for i_pattern in range(p['patts'].shape[0]):
        print(i_pattern, " ", end='')
        patt = p['patts'][i_pattern]
        pattLearnLen = patt.size * p['learnLength']
        xCollector = np.zeros((p['N'] + 1, pattLearnLen));
        xOldCollector = np.zeros((p['N'], pattLearnLen));
        WTargetCollector = np.zeros((p['N'], pattLearnLen));
        pCollector = np.zeros((1, pattLearnLen));
        x = np.zeros((p['N'], 1))
        tfWstar = tf.constant(Wstar)
        tfWin = tf.constant(Win)
        with tf.device(tfdevice):    

            for n in range(p['washoutLength'] + pattLearnLen):
                u = patt.take(n, mode='wrap')
                xOld = x
                Wtarget = (Wstar.dot(x)) + (Win.dot(u))
#                 wstarx=tf.matmul(tfWstar,x)
#                 winu = tf.multiply(tfWin,u)
#                 Wtarget = tf.add(wstarx, winu)

                leakTerm = LROneMinus.dot(xOld)
                newX =tanh(Wtarget + Wbias)
                newXLeaked = LR.dot(newX)
                x = leakTerm + newXLeaked
#                 xOldLR = tf.multiply(tf.constant(1.0-LR, dtype=float64), xOld)
#                 biasedTarget = tf.add(Wtarget, Wbias)
#                 biasedTarget = tf.tanh(biasedTarget)
#                 biasedTargetLR = tf.multiply(tf.constant(LR, dtype=float64), biasedTarget)
#                 x = tf.add(xOldLR, biasedTargetLR)

                if n >= p['washoutLength']:
                    xCollector[:, n - p['washoutLength']] = np.concatenate((x[:,0], np.array([1])))
                    xOldCollector[:, n - p['washoutLength']] = xOld[:,0]
                    WTargetCollector[:, n - p['washoutLength']] = Wtarget[:,0]
                    pCollector[0, n - p['washoutLength']] = u

                uOld = u

            R = xCollector[0:-1].dot(xCollector[0:-1].T) / pattLearnLen
            patternRs[0,i_pattern] = R
            allTrainxArgs = np.concatenate((allTrainxArgs, xCollector), axis=1)
            allTrainOldxArgs = np.concatenate((allTrainOldxArgs, xOldCollector), axis=1)
            allTrainOuts = np.concatenate((allTrainOuts, pCollector), axis=1)
            allTrainWtargets = np.concatenate((allTrainWtargets, WTargetCollector), axis=1)


    Wout = (linalg.inv(allTrainxArgs.dot(allTrainxArgs.conj().T) +
                      (p['TychonovAlphaReadout'] * np.eye(p['N'] + 1))).dot(allTrainxArgs).dot(allTrainOuts.conj().T)).conj().T

    outsRecovered = Wout.dot(allTrainxArgs);
    NRMSE_readout = mean(nrmse(outsRecovered, allTrainOuts))
    absSize_readout = mean(mean(abs(Wout), axis=0))
    print("\nNRMSE readout: ", NRMSE_readout, " :: ", end='')
    print("absSize readout: ", absSize_readout)
    
    W = (linalg.inv(allTrainOldxArgs.dot(allTrainOldxArgs.conj().T) +
                      (p['TychonovAlpha'] * np.eye(p['N']))).dot(allTrainOldxArgs).dot(allTrainWtargets.conj().T)).conj().T
    NRMSE_W = mean(nrmse(W.dot(allTrainOldxArgs), allTrainWtargets))
    absSize_W = mean(mean(abs(W), axis=0))
    print("NRMSE W: ", NRMSE_W, " :: ", end='')
    print("absSize W: ", absSize_W)

    data ={k: v for k, v in locals().items() if k in 
           ('p','Win','Wstar', 'Wbias','NRMSE_W', 'absSize_W','patternRs','W',
            'Wout','NRMSE_readout', 'absSize_readout')}
    return data

In [289]:

def render(p, patternCs, bestNet, lrMod=1.0, speed=1.0, xFade=0.05, srMod=1):
    audio = np.zeros(0)
    x = 0.5 * np.random.randn(p['N'],1)
    C = patternCs[0]
    
    LR = p['LR']
    LR = array(LR * lrMod)
    LROneMinus = array(1.0 - p['LR'])
    Wmod = bestNet['W'] * srMod
    
    #run to washout
    for n in range(p['washoutLength']):
        xOld = x
        Wtarget = (Wmod.dot(x))
        z = (LROneMinus.dot(xOld)) + (LR.dot(tanh(Wtarget + bestNet['Wbias'])))
        x = C.dot(z)


    for i_patt in range(p['patts'].shape[0]):
        xFadeTime=int(p['patts'][i_patt].shape[0] * xFade)
        for n in range(int(p['patts'][i_patt].shape[0] * speed)):
            C = patternCs[i_patt]
            v=int(p['patts'][i_patt].shape[0] * speed)
            stepL = min(v - n - 1, xFadeTime)
            stepU = min(n, xFadeTime)
            m1 = 1.0
            if(n > v-xFadeTime-1 and i_patt < p['patts'].shape[0]-1):
                m1 = (stepL + stepU) / (2*xFadeTime)
                nextC = patternCs[i_patt+1]
                C = (m1 * C) + ((1.0-m1) * nextC)
            else:
                if (n < xFadeTime and i_patt > 0):
                    m1 = 0.5 - (n / (2*xFadeTime))
                    prevC = patternCs[i_patt-1]
                    C = (m1 * prevC) + ((1.0-m1) * C)
#                 else:
#                     C = cNet['Cs'][0,0]
            
            xOld = x
            Wtarget = (Wmod.dot(x))
            z = (LROneMinus.dot(xOld)) + (LR.dot(tanh(Wtarget + bestNet['Wbias'])))
            x = C.dot(z)

            newSample = bestNet['Wout'].dot(np.concatenate((x[:,0], np.array([1]))))
            audio = np.concatenate((audio, newSample))
    return audio



In [294]:
def evalModel(genome, patterns, patternLengths, orgAudio, N=900):
    LR =genome[0]
    modelParams = {'N':N, 'NetSR':1.5, 'NetinpScaling':1.2,'BiasScaling':0.3, 'TychonovAlpha':0.0001,
             'washoutLength':50, 'learnLength':4, 'TychonovAlphaReadout':0.0001,
              'LR': LR,
              'patts':patterns
             }   
    newNetwork  = makeLoadedNetwork_v2(modelParams)
    with tf.device(tfdevice):
        tfWbias = tf.constant(newNetwork['Wbias'])
        tfW = tf.constant(newNetwork['W'])

    
    import scipy
    def fitnessf(aperture, *args):
        print('Pattern: ', args[0])
        params = args[1]
        net = args[2]
        try:
            C = computeConceptor(params, net, args[0], aperture)
        except:
            print("Exception when computing conceptor")
            return 999
        atten = testConceptor(params, C, net, params['patts'][args[0]].size * params['learnLength'], args[3], args[4])
        return atten

    apertures = [scipy.optimize.fminbound(fitnessf, 0,1000,  disp=2, xtol=15, args = (x,modelParams, newNetwork, tfW, tfWbias)) 
                 for x in np.arange(modelParams['patts'].shape[0])]

    #store conceptors with calculated apertures
    patternCs = np.zeros(len(apertures), dtype=np.object)
    for i_patt in range(patternCs.size):
        patternCs[i_patt] = computeConceptor(modelParams, newNetwork, i_patt, apertures[i_patt]).numpy()
        
#     figsize(20,3)
    audio = render(modelParams, patternCs, newNetwork, 1.0,1.0, 0.05)
    error = compareMFCCs(audio, orgAudio)

#     plot(audio)
    return {"error":error, "waveform":audio, 'apertures':apertures, 'net':newNetwork}


In [295]:
def evalFitness(genome, data):

    modelData = evalModel(genome, data['patterns'], data['patternLengths'], data['orgAudio'], data['N'])
    if ("winner" in data):
        if modelData['error'] < data['winner']['error']:
            data['winner'] = modelData
    else:
        data['winner'] = modelData
    return modelData['error']


In [ ]:
startTS = datetime.now()
testing = False
learningRates = linspace(0.05,0.95,10)
# for currentIxi in range(len(ixiFiles)):
for currentIxi in range(1) if testing else range(len(ixiFiles)):
    def log(msg):
        f = open(resultsFolderName + ixistr(currentIxi) + "searchLog.txt", "a")
        f.write(str(datetime.now()) + ":")
        f.write(msg)
        f.write('\r\n')
        f.close()  
        print(msg)
    clear_output()
    print("loading: ", ixiFiles[currentIxi])
    y, sr = librosa.load("audios/" + ixiFiles[currentIxi], sr=22050)
    y = y[:5000] / np.max(y) * 0.5
    print(sr)

    #divide out windows
    patterns = []

    minPatternSize = 9
    lastCrossing=0
    for i in range(y.shape[0]-1):
        if (i-lastCrossing) > minPatternSize and y[i] >=0 and y[i+1] < 0:
            print(i)
            segment = y[lastCrossing:i]
            patterns.append(segment)
            lastCrossing = i
            
    #convert to numpy
    patterns = np.array(patterns, dtype=np.object)
    maxPatterns = 150
    patterns = patterns[:maxPatterns]
    patternLengths = [x.shape[0] for x in patterns]
    y = y[:sum(patternLengths)]
    maxPatternLen = np.max(patternLengths)
    minPatternLen = np.min(patternLengths)
    print("max length: ", maxPatternLen)
    print("min length: ", minPatternLen)
    # for p in patterns:
    #     plot(p)
    print(patterns.shape)
    
    
    data = {'patterns':patterns, 'patternLengths':patternLengths, 'orgAudio':y}
    def onEpochStart():
        clear_output()
        print('Runtime:', print(datetime.now() - startTS))
        print(currentIxi, '/', len(ixiFiles), ' : ', ixiFiles[currentIxi])
        log(str(scores))

#     brute force search of learning rates
#     do the search at low res with smaller N
    data['N'] = 600
    scores = zeros_like(learningRates)
    for i,l in enumerate(learningRates):
        trials = 5
        trialScores = zeros(trials)
        for trial in range(trials):
            onEpochStart()
            log("lo res trial" + str(trial))
            log(str(scores))
            log(str(trialScores))
            trialScores[trial] = evalFitness(array([l]), data)
        scores[i] = np.median(trialScores)
        log(str(trialScores))
    
    winningScore = np.min(scores)
    log("Winning score: " + str(winningScore))
    bestLR = learningRates[np.argmin(scores)]
#     now generate the best of x at high res to find a good network
    data['N'] = 900
    trials = 10
    scores = zeros(trials)
    del data['winner']
    for trial in range(trials):
        onEpochStart()
        log("hi res trial " + str(trial))
        scores[trial] = evalFitness(array([bestLR]), data)
        log(str(scores))
        
        
    
    plt.close()
    figsize(20,4)
    plt.xlabel("Time (samples)", fontsize=20)
    plt.ylabel("Amplitude", fontsize=20)
    plt.xticks(fontsize=18, rotation=0)
    plt.yticks(fontsize=18, rotation=0) 
    plot(y, label='Original', alpha=0.6)
    plot(data['winner']['waveform'], alpha=1.0, label='Reconstruction')
    plt.legend(fontsize=20)
    plt.savefig(resultsFolderName + ixistr(currentIxi) + "compare.pdf", bbox_inches='tight')

    librosa.output.write_wav(resultsFolderName + ixistr(currentIxi) + "org.wav",y, sr)
    librosa.output.write_wav(resultsFolderName + ixistr(currentIxi) + "recon.wav", data['winner']['waveform'], sr)

    import dill as pickle
    with open(resultsFolderName + ixistr(currentIxi) + r"model.dill.pickled", "wb") as output_file:    
        pickle.dump({'winner':data['winner'], 'original':y, 'patterns':patterns}, output_file, protocol=0)
